In [1]:
!pip install -q transformers torch
from transformers import pipeline

model_name = "XerOpred/twitter-climate-sentiment-model"
classifier = pipeline('sentiment-analysis', model=model_name)

text = "some power and authority u can not spell, let alone define and wield, thas just more evidence of ur arrogant IGNORANCE same as u apply to ur climate change denial THEORY as if u know shit u do not, TOLD U DareDevil does not mean what the hell u think it does, HELL?? been there"
result = classifier(text)
print(result)

2023-12-06 02:00:53.739133: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[{'label': 'LABEL_1', 'score': 0.7463672161102295}]


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd

df = pd.read_csv('data/combined-usa.csv' )

model_name = "XerOpred/twitter-climate-sentiment-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def sentiment_analysis(model, tokenizer, text):
    # tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    
    # get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # transform output logits to probabilities using softmax
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # get the highest probability label
    predicted_label_idx = torch.argmax(probabilities, dim=1).item()
    labels = model.config.id2label

    return labels[predicted_label_idx], probabilities[0][predicted_label_idx].item()

df['Sentiment'], df['Confidence'] = zip(*df['Content'].apply(lambda x: sentiment_analysis(model, tokenizer, str(x))))
df.to_csv('data/distilbert-sentiment-usa.csv', index=False)
